In [1]:
import pandas as pd
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 2.6 MB/s eta 0:00:01


In [2]:
#import the library
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"

    return negative, neutral, positive, compound, overall_sentiment

In [72]:
import pandas as pd
data = pd.read_csv("final_tweets_list.csv")

In [17]:
sentiment = []
for each in data["tweets"].tolist():
    negative, neutral, positive, compound, overall_sentiment = sentiment_vader(each)
    sentiment.append(overall_sentiment)

In [73]:
data["sentiment"] = sentiment

In [74]:
data

,Unnamed: 0,event_id,tweets,sentiment
0,0,Georgia1,https://t.co/KwMLUjBum9\n\nThree jail guards i...,Negative
1,1,Georgia1,September Complaint: Voting machine testing co...,Negative
2,2,Georgia1,A man accused of fatally shooting another pers...,Negative
3,3,Georgia1,"Oops…\n\nGA Tech’s defense has been okay, but ...",Neutral
4,4,Georgia1,Scott Frost bragged about how his new O-Line c...,Negative
...,...,...,...,...
2235,179,Washington14,@Madonna I make intercessions with Heaven for ...,Positive
2236,180,Washington14,Remember when Eagles coach Nick Sirianni took ...,Positive
2237,181,Washington14,@SovCam @juliegreen52722 @GloryBound_info It w...,Neutral
2238,182,Washington14,@NWSportsfan1992 @Softykjr These polls are sup...,Negative


In [95]:
new_df = data.groupby('event_id', group_keys=True)["sentiment"].apply(list)

In [101]:
new_df

event_id
Boston1      [Positive, Negative, Positive, Positive, Posit...
Boston2      [Neutral, Neutral, Positive, Positive, Positiv...
Boston3      [Positive, Positive, Positive, Neutral, Neutra...
Boston4      [Positive, Neutral, Neutral, Negative, Positiv...
Boston5                          [Positive, Neutral, Negative]
                                   ...                        
illinois5    [Negative, Positive, Neutral, Neutral, Positiv...
illinois6    [Neutral, Positive, Positive, Negative, Positi...
illinois7    [Positive, Negative, Positive, Positive, Posit...
illinois8    [Neutral, Negative, Negative, Neutral, Negativ...
illinois9    [Positive, Positive, Negative, Positive, Posit...
Name: sentiment, Length: 163, dtype: object

In [98]:
max_val = []
keys = []
for key in new_df.keys():
    vals = new_df.get(key)
    keys.append(key)
    max_val.append(max(vals, key=vals.count))

In [104]:
final_sent = pd.DataFrame()
final_sent["event_id"] = keys
final_sent["event_sentiment"] = max_val

In [105]:
#new_df["overall_sentiment"] = max_val
final_sent

,event_id,event_sentiment
0,Boston1,Positive
1,Boston2,Positive
2,Boston3,Positive
3,Boston4,Positive
4,Boston5,Positive
...,...,...
158,illinois5,Negative
159,illinois6,Positive
160,illinois7,Positive
161,illinois8,Negative


In [107]:
org_data = pd.read_csv("final_dataset_v1.csv")

In [108]:
final_data_df = pd.merge(org_data, final_sent, on="event_id")

In [111]:
final_data_df.to_csv("final_dataset_v1.csv", index=False)